In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
save_check_pt = './checkpoints/weather'
my_epoch = 1

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = cv2.imread(img_dir)
        img = cv2.resize(img,(32,32))
        X.append(img)
    X = np.array(X)
    return X

def get_encoder(input_shape=(32,32,3)):
    net = keras.applications.ResNet50(include_top=False,input_shape= input_shape, pooling="avg")
    net.trainable = False
    newnet = keras.Sequential([
        net, # drop last layer ResNet50
        layers.Dense(512, activation='relu'), 
        layers.BatchNormalization(), 
        layers.Dropout(rate=0.5), 
        layers.Dense(35) # 35 category
    ])
    newnet.build(input_shape=input_shape)
    newnet.compile(optimizer=Adam(0.01), loss='CategoricalCrossentropy')
    return newnet

def get_task(input_shape=(2,)):
    model = Sequential()
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(0.01), loss='mse')
    return model

def get_discriminator(input_shape=(2,)):
    model = Sequential()
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(0.01), loss='mse')
    return model
base_images_dir, base_labels, base_table = load_ccpd(dataset_path['base'],mode='all')
base_images = readCcpdImg(base_images_dir)
base_labels = np.array(base_labels)
weather_images_dir, weather_labels, weather_table = load_ccpd(dataset_path['weather'],mode='all')
weather_images = readCcpdImg(weather_images_dir)
weather_labels = np.array(weather_labels)
weather_images_dir_t, weather_labels_t, weather_table_t = load_ccpd(dataset_path['weather'],mode='train')
weather_images_t = readCcpdImg(weather_images_dir_t)
weather_labels_t = np.array(weather_labels_t)
print(np.shape(base_images))
print(np.shape(weather_images))
print(np.shape(weather_images_t))
model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.001),  metrics=["acc"],random_state=0)
model.fit(base_images, base_labels,weather_images_t, epochs=my_epoch,verbose=1,batch_size = 32)
model.summary()
model.score(weather_images, weather_labels)
model.save_weights(save_check_pt)

(8446, 32, 32, 3)
(3812, 32, 32, 3)
(762, 32, 32, 3)
286/286 [==============================] - 24s 66ms/step - loss: 155.5574 - acc: 0.0404 - disc_loss: 4.0601 - disc_acc: 0.4755
Model: "dann"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Sequential)        (None, 35)                24656803  
                                                                 
 task (Sequential)           (None, 1)                 371       
                                                                 
 discriminator (Sequential)  (None, 1)                 371       
                                                                 
Total params: 24,657,545
Trainable params: 1,068,809
Non-trainable params: 23,588,736
_________________________________________________________________
1/1 [==============================] - 3s 3s/step - loss: 160.8657 - acc: 0.0359


In [2]:
# fig = plt.figure()
# img_h, img_w = 32,16
# fig.subplots_adjust(hspace=0.4, wspace=0.4)
# for i in range(6):
#     ax = fig.add_subplot(2, 3, i+1)
#     ax.imshow(weather_images[i].reshape([img_h, img_w]))
#model.save_weights('ckpt')
#dummy step to load model from checkpoint
#xt = np.zeros((3812, 512))
#nmd = DANN(lambda_=0.1, Xt=xt, metrics=["acc"], random_state=0)
#x = np.zeros((1,1))
#y = np.zeros(1)
#nmd.fit(x,y, epochs=0,verbose=1)
#nmd.summary()
#nmd.load_weights("ckpt")
#nmd.score(weather_images, weather_labels)
#print(nmd.predict(weather_images[0].reshape([-1,512])))